# Mutual Fund Investment Analysis

## Python (Flask) App, majorly for Backend APIs for Individual Mutual Fund Investment Analysis

### Problem Statement

Over the years, investors have tried and tested various methodologies to keep a track of all investements. The various problems faced for the same are -

1. Multi-vendor - The Mutual Funds are invested via multiple vendors, like Groww, Paytm Money, etc., hence no unified interface to track all investments.
2. The historic data is not accurately predicted - Over time the historic data loses importance and is overriden to be fitted only in Time Series Graph.
3. Periodic Tracking of data - Everyday net change in the invested amount.
4. Personalized Prediction - Currently all Mutual Fund Predictions are not personalized, only based on overall NAV changes.

### Data collection, Storage and Analysis Blueprint

Data from differen apps will be collected in the following way -

1. Per day data of return is manually entered in Google Sheet.
2. The data from the Google Sheet is fetched in Python, and stored in a MySQL DB.
3. Data fetching happens everyday, at 11:00 a.m. (APScheduler)
4. Success or failure mails for everyday update is triggered based on the storage of data in the respective database.
5. APIs expose the various data, grouped by various factors to be used in the UI.
6. Once the stored data crosses a significant volume, this data is splitted into train and test data for future analysis.
7. The predicted data is again exposed over APIs, grouped by various factors.

## Connecting to Google Sheet

Our primary data source is google sheet, where the daily changes for all the mutual funds, are recorded.

To connect the Google Sheet, we perform the following -

1. Go to https://console.cloud.google.com/ and create a new Project.
2. In the created project, enable Google Drive API
3. Create credentials to access the Google Drive API.
4. Enable the Google Sheets API
5. Share the Google Sheet with the dev ID generated in the credential

### All imports at one place

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from datetime import datetime,timedelta
import smtplib
from email.message import EmailMessage
import json
import numpy as np

/usr/lib/python3/dist-packages/Crypto/Random/Fortuna/FortunaGenerator.py:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sys.version_info[0] is 2 and  sys.version_info[1] is 1:
/usr/lib/python3/dist-packages/Crypto/Random/Fortuna/FortunaGenerator.py:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sys.version_info[0] is 2 and  sys.version_info[1] is 1:


In [3]:
#Defining the scope of the OAuth Authentication
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
         
#Getting the credentials
creds = ServiceAccountCredentials.from_json_keyfile_name("./flask_app/src/secret_config/google_credentials.json", scope)
#Connecting to the Google Spreadsheet Client
client = gspread.authorize(creds)

#Getting the spreadsheet
sheet = client.open("Daily_MF_Returns").sheet1


### Getting all the data from the sheet

Explore the sheet data

In [7]:
list_data = sheet.get_all_records()

#Creating the dataframe
data = pd.DataFrame(list_data)

In [8]:
#Rows and Column Count for the data
data.shape

(130, 6)

In [9]:
#Column Names of the data
data.columns

Index(['Date', 'Policy Name', 'App', 'Investment', 'Return', 'Net Change'], dtype='object')

In [10]:
#explore the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         130 non-null    object
 1   Policy Name  130 non-null    object
 2   App          130 non-null    object
 3   Investment   130 non-null    int64 
 4   Return       130 non-null    object
 5   Net Change   130 non-null    object
dtypes: int64(1), object(5)
memory usage: 6.2+ KB


In [1]:
data.tail(5).values

NameError: name 'data' is not defined

### Replacing Empty Values

In [14]:
data.replace([''],'Unknown',inplace=True)

### Adding Formatted Date column to the dataframe

The new column contains date in format DD-MMM-YYYY to avoid ambiguity

In [50]:
# Coverting all the values in proper Datetime format

def validate(date_text):
    try:
        return datetime.strptime(date_text, '%m/%d/%Y').strftime("%d-%b-%Y")
    except ValueError:
        print(date_text)
        return date_text
    
data['date_formatted'] = data['Date'].apply(validate)

## Update the Excel for next week (repetitive)

### Get the last date, and all the records for last date
We'll update the data for next week (Tuesday - Saturday), every previous Saturday
To do this the following has to be calculated -

1. Index of the current pointer in the excel
2. The last date
3. All the records for the last date (fetch Date, Policy Name, App and Investment)

In [51]:
current_pointer = len(list_data)

#Getting the last date
last_date = data.values[current_pointer-1][0]

#Getting all the rows corresponding to the last date
data_filtered = data.loc[data['Date'] == last_date]

data_filtered = data_filtered.drop(['Return','Net Change', 'date_formatted'],axis=1)

# List of dictionaries
list_to_insert = data_filtered.values.tolist()

print(list_to_insert)

[['11/28/2020', 'IDFC Low Duration', 'Paytm Money', 1600], ['11/28/2020', 'BOI AXA', 'Paytm Money', 200], ['11/28/2020', 'SBI Short Term', 'Paytm Money', 1000], ['11/28/2020', 'Edelweiss Banking and PSU', 'Paytm Money', 500], ['11/28/2020', 'HDFC Gold Direct', 'Paytm Money', 500], ['11/28/2020', 'Nippon India Liquid Fund', 'Paytm Money', 800], ['11/28/2020', 'ICICI Prudential Regular Gold', 'Groww', 500], ['11/28/2020', 'Axis Midcap Direct', 'Groww', 500], ['11/28/2020', 'Axis Bluechip', 'Groww', 500], ['11/28/2020', 'SBI Magnum', 'Groww', 500]]


### Create the final list of data to be inserted

1. Start from 2 days from the date
2. Create for 5 days
3. Total # rows = Current # rows * 5

In [52]:
final_insert_list = []

for i in range(0,5):
    for each_row in list_to_insert:        
        final_insert_list.append(each_row)

# Yield successive n-sized 
# chunks from l. 
def divide_chunks(l, n):       
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n]
        
days_chunk = list(divide_chunks(final_insert_list, len(list_to_insert)))

## Update Records and Send Mail

1. Update the next weeks data in the sheet
2. Send success and failure mails accordingly

In [53]:
def email_alert(subject, body, to):
    msg=EmailMessage()
    msg.set_content(body)
    msg['subject']=subject
    msg['to'] = to
    
    gmail_credentials_file = open("./flask_app/src/secret_config/gmail_credentials.json")
    gmail_credentials = json.load(gmail_credentials_file)
    
    sender = gmail_credentials['email']
    password = gmail_credentials['password']    
    
    msg['from'] = sender
    
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(sender,password)
    
    server.send_message(msg)
    server.quit()
    print("Email sent successfully");

success_insert = True
total_rows = data.shape[0] + 2 # 1st Row is heading

days_delta = 3

for each_chunk in days_chunk:
    if(success_insert):
        date_to_add = (datetime.strptime(last_date, '%m/%d/%Y') + timedelta(days=days_delta)).strftime('%m/%d/%Y')
        for value in each_chunk:
            value[0] = date_to_add
            try:
                sheet.insert_row(value, total_rows, 'RAW')
                print("Updated Row - " + str(total_rows))
                total_rows += 1                
            except:
                print("Error in Row - " + str(total_rows))
                success_insert = False
                break
        days_delta +=1
    else:
        break

start_date = (datetime.strptime(last_date, '%m/%d/%Y') + timedelta(days=3)).strftime('%d-/%b-%Y')

if(success_insert):
    body = '''
    Hi Apratim,
    
    The weekly scheduled insert of data is successful. Data is inserted for the next 5 days, starting from Tuesday - {var}.
    Please update the returns accordingly.
    
    Best Regards,
    Dev Team
    Mutual Fund Analysis App'''.format(var=start_date)
else:
    body = '''
    Hi Apratim,
    
    There was some issue in updating your data. Rest assured our team is working on it.
    Meanwhile please update the data and returns manually, starting from - {var}.
    Sorry for the incovenience caused.
    
    Best Regards,
    Dev Team
    Mutual Fund Analysis App'''.format(var=start_date)
    
subject = "Mutual Funds - Weekly Insertion of Base Data"
to = "apratimnath7@gmail.com";

email_alert(subject, body, to)

Updated Row - 82
Updated Row - 83
Updated Row - 84
Updated Row - 85
Updated Row - 86
Updated Row - 87
Updated Row - 88
Updated Row - 89
Updated Row - 90
Updated Row - 91
Updated Row - 92
Updated Row - 93
Updated Row - 94
Updated Row - 95
Updated Row - 96
Updated Row - 97
Updated Row - 98
Updated Row - 99
Updated Row - 100
Updated Row - 101
Updated Row - 102
Updated Row - 103
Updated Row - 104
Updated Row - 105
Updated Row - 106
Updated Row - 107
Updated Row - 108
Updated Row - 109
Updated Row - 110
Updated Row - 111
Updated Row - 112
Updated Row - 113
Updated Row - 114
Updated Row - 115
Updated Row - 116
Updated Row - 117
Updated Row - 118
Updated Row - 119
Updated Row - 120
Updated Row - 121
Updated Row - 122
Updated Row - 123
Updated Row - 124
Updated Row - 125
Updated Row - 126
Updated Row - 127
Updated Row - 128
Updated Row - 129
Updated Row - 130
Updated Row - 131
Email sent successfully


## Calculations (Transferred via API)

API 1 - Get the daily change based on the following parameters -
1. Overall
2. App Based
3. Fund Based

In [65]:
#Subset of data containing returns
valid_data_with_returns = data.loc[data['Return'] != 'Unknown']

length_of_valid_returns = valid_data_with_returns.shape[0]

#Getting the last date
last_date = valid_data_with_returns.values[length_of_valid_returns-1][0]

#Get the last date and the date before that data
last_date_rows = valid_data_with_returns.loc[valid_data_with_returns['Date'] == last_date]
previous_date_rows = valid_data_with_returns.loc[valid_data_with_returns['Date'] != last_date].tail(last_date_rows.shape[0])

# last_date_rows_list = last_date_rows.values.tolist()
# previous_date_rows_list = previous_date_rows.values.tolist()

#Convert to numeric
last_date_rows['Investment'] = pd.to_numeric(last_date_rows['Investment'])
last_date_rows['Return'] = pd.to_numeric(last_date_rows['Return'])
last_date_rows['Net Change'] = pd.to_numeric(last_date_rows['Net Change'])

previous_date_rows['Investment'] = pd.to_numeric(previous_date_rows['Investment'])
previous_date_rows['Return'] = pd.to_numeric(previous_date_rows['Return'])
previous_date_rows['Net Change'] = pd.to_numeric(previous_date_rows['Net Change'])

<ipython-input-65-30f27d17b758>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_date_rows['Investment'] = pd.to_numeric(last_date_rows['Investment'])
<ipython-input-65-30f27d17b758>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_date_rows['Return'] = pd.to_numeric(last_date_rows['Return'])
<ipython-input-65-30f27d17b758>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

### App Wise Calculation

In [71]:
#App-wise calculation
grouped_last = last_date_rows.groupby('App',as_index=False).agg({'Net Change':[np.sum, np.mean, np.std]})
grouped_previous = previous_date_rows.groupby('App',as_index=False).agg({'Net Change':[np.sum, np.mean, np.std]})

#Rename previos frame colums
grouped_previous.columns = ['app','prev_sum','prev_mean','prev_std']


#Create single comparision frame
compare_frame = grouped_last
compare_frame.columns = ['app','current_sum','current_mean','current_std']

compare_frame['prev_sum'] = grouped_previous['prev_sum']
compare_frame['prev_mean'] = grouped_previous['prev_mean']
compare_frame['prev_std'] = grouped_previous['prev_std']

compare_frame['sum_diff'] = np.where(compare_frame['current_sum'] == compare_frame['prev_sum'], 0 , compare_frame['current_sum'] - compare_frame['prev_sum'])
compare_frame['mean_diff'] = np.where(compare_frame['current_mean'] == compare_frame['prev_mean'], 0 , compare_frame['current_mean'] - compare_frame['prev_mean'])
compare_frame['std_diff'] = np.where(compare_frame['current_std'] == compare_frame['prev_std'], 0 , compare_frame['current_std'] - compare_frame['prev_std'])

compare_frame = compare_frame.drop(['current_sum','current_mean', 'current_std', 'prev_sum','prev_mean', 'prev_std'],axis=1)

app_diff_list = compare_frame.values.tolist()

app_dict_list = []
for each_app in app_diff_list:
    current_dict = {}
    current_dict['app_name'] = each_app[0]
    current_dict['sum_difference'] = round(each_app[1],2)
    current_dict['mean_difference'] = round(each_app[2],2)
    current_dict['standard_deviation_difference'] = round(each_app[3],2)
    
    app_dict_list.append(current_dict)

print(app_dict_list)

[{'app_name': 'Groww', 'sum_difference': 8.0, 'mean_difference': 2.0, 'standard_deviation_difference': 4.7}, {'app_name': 'Paytm Money', 'sum_difference': 0.49, 'mean_difference': 0.08, 'standard_deviation_difference': -0.39}]


### Fund Based Calculation

Since the values are independent, ideally group by has no effect.

However, we keep on using the costlier grouping to keep things inline with the previous implementation

In [77]:
#Fund-wise calculation
grouped_last = last_date_rows.groupby('Policy Name',as_index=False).agg({'Net Change':[np.sum]})
grouped_previous = previous_date_rows.groupby('Policy Name',as_index=False).agg({'Net Change':[np.sum]})

#Rename previos frame colums
grouped_previous.columns = ['policy_name','prev_sum']


#Create single comparision frame
compare_frame = grouped_last
compare_frame.columns = ['policy_name','current_sum']

compare_frame['prev_sum'] = grouped_previous['prev_sum']

compare_frame['sum_diff'] = np.where(compare_frame['current_sum'] == compare_frame['prev_sum'], 0 , compare_frame['current_sum'] - compare_frame['prev_sum'])

compare_frame = compare_frame.drop(['current_sum', 'prev_sum'],axis=1)

policy_diff_list = compare_frame.values.tolist()

policy_dict_list = []
for each_policy in policy_diff_list:
    current_dict = {}
    current_dict['policy_name'] = each_policy[0]
    current_dict['sum_difference'] = round(each_policy[1],2)
    
    policy_dict_list.append(current_dict)

print(policy_dict_list)

[{'policy_name': 'Axis Bluechip', 'sum_difference': -1.0}, {'policy_name': 'Axis Midcap Direct', 'sum_difference': 10.0}, {'policy_name': 'BOI AXA', 'sum_difference': 2.82}, {'policy_name': 'Edelweiss Banking and PSU', 'sum_difference': 0.1}, {'policy_name': 'HDFC Gold Direct', 'sum_difference': -2.11}, {'policy_name': 'ICICI Prudential Regular Gold', 'sum_difference': -1.0}, {'policy_name': 'IDFC Low Duration', 'sum_difference': 0.09}, {'policy_name': 'Nippon India Liquid Fund', 'sum_difference': 0.06}, {'policy_name': 'SBI Magnum', 'sum_difference': 0.0}, {'policy_name': 'SBI Short Term', 'sum_difference': -0.47}]


### Overall Calculation

In [79]:
#Overall calculation
grouped_last = last_date_rows.groupby('Date',as_index=False).agg({'Net Change':[np.sum, np.mean, np.std]})
grouped_previous = previous_date_rows.groupby('Date',as_index=False).agg({'Net Change':[np.sum, np.mean, np.std]})

#Rename previos frame colums
grouped_previous.columns = ['date','prev_sum','prev_mean','prev_std']


#Create single comparision frame
compare_frame = grouped_last
compare_frame.columns = ['date','current_sum','current_mean','current_std']

compare_frame['prev_sum'] = grouped_previous['prev_sum']
compare_frame['prev_mean'] = grouped_previous['prev_mean']
compare_frame['prev_std'] = grouped_previous['prev_std']

compare_frame['sum_diff'] = np.where(compare_frame['current_sum'] == compare_frame['prev_sum'], 0 , compare_frame['current_sum'] - compare_frame['prev_sum'])
compare_frame['mean_diff'] = np.where(compare_frame['current_mean'] == compare_frame['prev_mean'], 0 , compare_frame['current_mean'] - compare_frame['prev_mean'])
compare_frame['std_diff'] = np.where(compare_frame['current_std'] == compare_frame['prev_std'], 0 , compare_frame['current_std'] - compare_frame['prev_std'])

compare_frame = compare_frame.drop(['current_sum','current_mean', 'current_std', 'prev_sum','prev_mean', 'prev_std'],axis=1)

overall_diff_list = compare_frame.values.tolist()

overall_dict_list = []
for each_overall in overall_diff_list:
    current_dict = {}
    current_dict['date'] = each_overall[0]
    current_dict['sum_difference'] = round(each_overall[1],2)
    current_dict['mean_difference'] = round(each_overall[2],2)
    current_dict['standard_deviation_difference'] = round(each_overall[3],2)
    
    overall_dict_list.append(current_dict)

print(overall_dict_list)

[{'date': '11/28/2020', 'sum_difference': 8.49, 'mean_difference': 0.85, 'standard_deviation_difference': -0.53}]
